From from this article: [Time Series — ARIMA vs. SARIMA vs. LSTM: Hands-On Tutorial](https://towardsdatascience.com/time-series-arima-vs-sarima-vs-lstm-hands-on-tutorial-bd5630298da3)

First we want to start off with downloading the data. It can be had from [here](https://archive.ics.uci.edu/dataset/360/air+quality). It's around 1 MB.

In [ ]:
# import libraries
import pandas as pd
import numpy as np

from ucimlrepo import fetch_ucirepo

# Fetch dataset
air_quality = fetch_ucirepo(id=360)

Get the data from the data set.

Same thing with many names:
- Features, X, input variables, independent variables
- Targets, y, output variables, dependent variables

In [ ]:
# Data (as pandas dataframe)
air_quality_df = air_quality.data.features
air_quality_df.head()

Explore the data a bit.

In [ ]:
air_quality_df.info()

See statistics of the data.

In [ ]:
air_quality_df.describe()

From the documentation for this data: Missing values are tagged with -200 value. Let's
see how many are -200 in each column. This can give us an idea of how to handle them.

In [ ]:
# Do an element wise comparison on all values, then sum up for each column.
missing_values_count = (air_quality_df == -200).sum()
missing_values_count

This is great! We see how many values are effectively null. Let's get a percentage too
so we can see for each column what the percentage of null (-200) there is.

In [ ]:
# Calculate the percentage of missing values for each column. Rount to whole int
missing_values_percentage = ((missing_values_count / len(air_quality_df)) * 100).round().astype(int)
missing_values_percentage

<!--TODO on second pass: For the readings, let's use the average value of the column to fill in the missing values. -->
## Feature Engineering
We'll handle missing values by dropping them.

In [ ]:
# Show column count and row count for before and after dropping missing values
air_quality_df.shape

In [ ]:
# Replace missing values and drop unnecessary columns.

# Replace -200 with NaN
air_quality_df.replace(-200, np.nan, inplace=True)
# Drop columns where all values are NaN
# - axis=1 means columns, how='all' means only drop if all values in the column are NaN
air_quality_df.dropna(axis=1, how='all', inplace=True)
air_quality_df.shape


In [ ]:
# Drop any rows with missing values.
air_quality_df.dropna(inplace=True)
air_quality_df.shape

Not great, we lost most the data set. I don't like how the tutorial handled this.
I think we can do better. TODO: come back to this

Now let's combine the Date and Time columns into a single datetime column.

In [ ]:
# Combine Date and Time columns into a single datetime column
air_quality_df['DateTime'] = pd.to_datetime(air_quality_df['Date'] + ' ' + air_quality_df['Time'])
air_quality_df.head()

We can set the new datetime column as the index since we're working with time series
data, then sort the data by the index.
Note 💡: Inplace is more memory efficient than having it return one and then re-assigning
it back to the variable.

In [ ]:
air_quality_df.set_index('DateTime', inplace=True)
air_quality_df.sort_index(inplace=True)

The tutorial picks Nox(GT) as the target variable without a reason why, so we'll use
that too, but don't ask me why. 😅

In [ ]:
# Grab the target column.
# Note 💡: This also grabs the index column, so you'll have a df with two columns.
target = air_quality_df['NOx(GT)']
target.head()

## Visualize
Let's visualize the data to see what we're working with. Visualizing is great. Always
visualize. ✨📊

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(target)
plt.title('Hourly NOx(GT) Levels')
plt.xlabel('Date')
plt.ylabel('NOx(GT) Concentration')
plt.show()



## Train Test Split
80/20 split. We'll use the last 20% of the data as the test set.
Note 💡: When the index for the data is an integer we can do normal slicing. But when
the index is an object like a date, which is what we have here, we need to use iloc
instead.

In [ ]:
# Get the index for where 80% falls.
train_size = int(len(target) * 0.8)


train, test = target.iloc[:train_size], target.iloc[train_size:]
train.shape, test.shape

# Modeling
Now to the fun part! We can start digging into making models.

Some definitions:
- Stationary: Mean, variance, and autocorrelation do not change over time. Autocorrelation is a value compared to a previous value. Often referred to as the lag. The lag is a time step and how many of that time step we're going to compare a given value to in the past. A good example of a stationary data set would be the temperature in a room set by a thermostat. The thermostat is set to something like 74˚F and the temperature will fluctuate around that value. The mean, variance, and autocorrelation will not change over time.
- Non-Stationary: Mean, variance, and autocorrelation do change over time. When they change, there's some specific terms for the ways they change.
  - Trend: The average changes over time. For example the cost of living goes up overtime because of inflation.
  - Seasonality: The variance changes over time, usually on a regular cadence. Like sales that are affected by the time of year like around holidays and the like.
  - Cyclical: The autocorrelation changes over time. This is a repeating pattern that occurs at irregular intervals.

We want to make non-stationary data into stationary data. There's some ways to do this, we'll use differencing here. Differencing is subtracting the previous value from the current value. This aims to remove trends from the data.

Some ways exist to tell if a dataset needs differencing. We'll use ADF (Augmented Dickey-Fuller) test. This test looks for Mean Reversion vs. Random Walk. In a stationary series, values may fluctuate, but revert to ta stable mean. In a non-stationary series, it will act more like someone walking at random where the next step doesn't depend on the previous step.
With this, if the p-value is less than a threshold, say 5%, then the dataset is stationary. If it's greater than 5%, then the dataset is non-stationary and we should difference it.

In [ ]:
from statsmodels.tsa.stattools import adfuller

# ADF Test
adf_result = adfuller(train)

# Pull the p-value and make it have 1 decimal place
p_value = (adf_result[1]*100).round(1)

print(f'P-Value: {p_value}%' + (' (Stationary)' if p_value < 5 else ' (Non-Stationary)'))

We'll now difference the data. Each time we difference it, it is one order of
differencing. So this is a first order differencing. You can difference it more but
doing it more than twice, 3rd order differencing, and higher generally ends up hurting
more than helping and ruins the data. So we'll do it once, maybe twice.

In [ ]:
# First-order differencing
train_diff = train.diff()

# Remove the first NaN value
train_diff.dropna(inplace=True)
train_diff


In [ ]:

# Check ADF p-value again
adf_result = adfuller(train_diff)
p_value = (adf_result[1]*100).round(1)

print(f'P-Value: {p_value}%' + (' (Stationary)' if p_value < 5 else ' (Non-Stationary)'))

Nice! The data is now stationary. We can now move on to the next step.


## ARIMA

Autoregressive Integrated Moving Average. ARIMA is a univariate linear model. It does not support seasonality in data. ARIMA parameters are: ARIMA(p, d, q). These three parameters account for seasonality, p, trend, d, and noise, q, in data.
  - p: It indicates the number of past values (lags) of the time series that the model will use to predict the current value. p=1 means that the model will use the last value to predict the current value, whicle p=2 means that the model will use the last two values to predict the current value, and so on. These are the order of the autoregressive, AR, part of the model.
  - d: It indicates the number of differences that the model will use to make the time series stationary. d=1 means that the model will use the first difference of the time series, d=2 means that the model will use the second difference of the time series, and so on. The order of integration, I, part of the model.
  - q: The number of previous forecast errors to use to adjust the current forecast prediction. The moving average, MA, part of the model. q=1 means that the model will use the last forecast error to adjust the current forecast, q=2 means that the model will use the last two forecast errors to adjust the current forecast, and so on.

We'll use Darts as our framework for all the models.

It's good to know what the parameters are and how they are calculated once we're comfortable with that, we can just use the auto finders to find the best values.

In [ ]:
from darts import TimeSeries
from darts.models import AutoARIMA

# Convert the pandas series to a Darts TimeSeries
darts_train = TimeSeries.from_series(train)

# Create the model
model = AutoARIMA(
    start_p=1,
    start_q=1,
    max_p=5,
)

